<a href="https://colab.research.google.com/github/shiri9/non-iid/blob/main/label_skew_statitistical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from google.colab import drive

# Mount Google Drive to access data files
drive.mount('/content/drive')

# Load datasets
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kdd_train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kdd_test.csv')

# Define label mapping for attack categories (including all labels from train and test sets)
attack_mapping = {
    'normal': 0, 'neptune': 1, 'land': 1, 'back': 1, 'teardrop': 1, 'pod': 1, 'smurf': 1,
    'ipsweep': 2, 'nmap': 2, 'portsweep': 2, 'satan': 2,
    'mailbomb': 1, 'apache2': 1, 'processtable': 1,  # Missing DoS labels in test set
    'phf': 3, 'multihop': 3, 'warezclient': 3, 'warezmaster': 3, 'spy': 3, 'ftp_write': 3,
    'guess_passwd': 3, 'imap': 3,
    'buffer_overflow': 4, 'loadmodule': 4, 'perl': 4, 'rootkit': 4,
    # Ensure all test labels are included
    'mscan': 2, 'saint': 2, 'snmpgetattack': 3, 'snmpguess': 3, 'xlock': 3, 'xsnoop': 3,
    'httptunnel': 3, 'ps': 4, 'xterm': 4,
    'sendmail': 3, 'named': 3  # Missing labels in test set
}

# Apply the label mapping
df_train['labels'] = df_train['labels'].replace(attack_mapping)
df_test['labels'] = df_test['labels'].replace(attack_mapping)

# Verify the unique labels after mapping
print("Unique labels in train set:", df_train['labels'].unique())
print("Unique labels in test set:", df_test['labels'].unique())

# Dropping the irrelevant column 'num_outbound_cmds'
df_train = df_train.drop('num_outbound_cmds', axis=1)
df_test = df_test.drop('num_outbound_cmds', axis=1)

# Encoding categorical columns: 'protocol_type', 'service', 'flag'
categorical_columns = ['protocol_type', 'service', 'flag']
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col])
    df_test[col] = le.transform(df_test[col])  # Important: use transform for test set, not fit_transform

# Scaling numerical columns
numerical_columns = [
    'duration', 'src_bytes', 'dst_bytes', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'same_srv_rate',
    'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
    'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_srv_diff_host_rate', 'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate', 'hot', 'num_compromised', 'num_root'
]

scaler = MinMaxScaler()
df_train[numerical_columns] = scaler.fit_transform(df_train[numerical_columns])
df_test[numerical_columns] = scaler.transform(df_test[numerical_columns])

# Convert to NumPy arrays and enforce correct types for TensorFlow
X_train = np.array(df_train.drop('labels', axis=1)).astype(np.float32)
y_train = np.array(df_train['labels']).astype(np.int32)

X_test = np.array(df_test.drop('labels', axis=1)).astype(np.float32)
y_test = np.array(df_test['labels']).astype(np.int32)

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

# Check dataset shapes
print("Train dataset shape:", X_train.shape, y_train.shape)
print("Test dataset shape:", X_test.shape, y_test.shape)


In [ ]:
#cell2
import numpy as np
import pandas as pd
import tensorflow as tf

# Number of partitions (clients)
num_partitions = 10

# Define minimum number of samples to ensure visibility for each class
min_samples_per_class = 50  # Set this to a value that ensures visibility

# Get unique classes/labels in the dataset (assuming 5 classes)
unique_labels_noniid = df_train['labels'].unique()
num_classes = len(unique_labels_noniid)

# Assign classes to each client. Adjusted for 5 classes
# Each client will observe a subset of the available classes
client_class_map = {
    0: [0, 1],  # Client 1 observes classes 0 and 1
    1: [0, 2],  # Client 2 observes classes 0 and 2
    2: [0, 3],  # Client 3 observes classes 0 and 3
    3: [0, 4],  # Client 4 observes classes 0 and 4
    4: [1, 2],  # Client 5 observes classes 1 and 2
    5: [1, 3],  # Client 6 observes classes 1 and 3
    6: [1, 4],  # Client 7 observes classes 1 and 4
    7: [2, 3],  # Client 8 observes classes 2 and 3
    8: [2, 4],  # Client 9 observes classes 2 and 4
    9: [3, 4]   # Client 10 observes classes 3 and 4
}

# Initialize list to store partitions
data_partitions = []

# For each partition (client), we only sample data from the assigned classes
for i in range(num_partitions):
    partition = pd.DataFrame()

    # Get the classes this client is supposed to observe
    client_classes = client_class_map[i]

    # Iterate through each class assigned to the current client
    for label in client_classes:
        class_data = df_train[df_train['labels'] == label]

        # Generate a random proportion for the current partition and class
        proportion = np.random.uniform(0.05, 0.5)  # Random proportion between 5% and 50%

        # Ensure at least `min_samples_per_class` samples are included for each class
        num_samples = max(min_samples_per_class, int(len(class_data) * proportion))

        # Randomly sample this number of instances from the class data
        sampled_data = class_data.sample(n=num_samples, replace=False)

        # Append the sampled data to the current partition
        partition = pd.concat([partition, sampled_data])

    # Shuffle the partition data and reset the index
    partition = partition.sample(frac=1).reset_index(drop=True)

    # Add the partition to the list
    data_partitions.append(partition)

    # Display class distribution in this partition
    print(f"Partition {i+1} class distribution:")
    print(partition['labels'].value_counts())
    print()


In [ ]:

# ## Cell3: Create Label-Skew Partitions

import numpy as np
import pandas as pd

# Configuration
NUM_CLIENTS = 10
CLASS_MAPPING = {'Benign': 0, 'DoS': 1, 'Probe': 2, 'U2R': 3, 'R2L': 4}
MIN_SAMPLES_PER_CLASS = 50  # Prevent class starvation

# Label distribution per client (matches your paper's setup)
client_class_map = {
    0: ['Benign', 'DoS'],
    1: ['Benign', 'Probe'],
    2: ['Benign', 'U2R'],
    3: ['Benign', 'R2L'],
    4: ['DoS', 'Probe'],
    5: ['DoS', 'U2R'],
    6: ['DoS', 'R2L'],
    7: ['Probe', 'U2R'],
    8: ['Probe', 'R2L'],
    9: ['U2R', 'R2L']
}

data_partitions = []
for client_id in range(NUM_CLIENTS):
    client_partition = pd.DataFrame()
    classes = client_class_map[client_id]

    for class_name in classes:
        label = CLASS_MAPPING[class_name]
        class_data = df_train[df_train['labels'] == label]

        # Dynamic sampling with minimum guarantee
        proportion = np.random.uniform(0.1, 0.4)  # 10-40% of class data
        num_samples = max(MIN_SAMPLES_PER_CLASS, int(len(class_data) * proportion))

        client_partition = pd.concat([
            client_partition,
            class_data.sample(n=num_samples, random_state=42+client_id)
        ])

    # Shuffle and store
    data_partitions.append(
        client_partition.sample(frac=1, random_state=42).reset_index(drop=True)
    )

    # Verification
    print(f"\nClient {client_id+1} Distribution:")
    print(client_partition['labels'].value_counts().sort_index())

In [ ]:

# ## Cell4 : Create TensorFlow Datasets (Final Corrected Version)

import numpy as np
import tensorflow as tf

# Configuration
batch_size = 32
SEED = 42  # For reproducible shuffling

train_datasets = []
val_datasets = []

for client_id, partition in enumerate(data_partitions):
    # ========== Exact 90/10 Split ==========
    total_samples = len(partition)
    train_samples = (total_samples // 10) * 9  # Exact 90%
    val_samples = total_samples - train_samples  # Exact 10%

    # Shuffle with client-specific seed
    shuffled_partition = partition.sample(frac=1, random_state=SEED+client_id).reset_index(drop=True)

    # Split into train/val
    train_part = shuffled_partition.iloc[:train_samples]
    val_part = shuffled_partition.iloc[train_samples:]

    # ========== Feature/Label Conversion ==========
    # Training data
    train_features = train_part.drop(columns=['labels']).values.astype(np.float32)
    train_labels = train_part['labels'].values.astype(np.int32)

    # Validation data
    val_features = val_part.drop(columns=['labels']).values.astype(np.float32)
    val_labels = val_part['labels'].values.astype(np.int32)

    # ========== Dataset Creation ==========
    train_dataset = tf.data.Dataset.from_tensor_slices(
        (train_features, train_labels)
    ).batch(batch_size)

    val_dataset = tf.data.Dataset.from_tensor_slices(
        (val_features, val_labels)
    ).batch(batch_size)

    # Store datasets
    train_datasets.append(train_dataset)
    val_datasets.append(val_dataset)

    # ========== Verification ==========
    print(f"Client {client_id+1}:")
    print(f"  Train: {len(train_part)} samples | Classes: {np.unique(train_labels)}")
    print(f"  Val: {len(val_part)} samples | Classes: {np.unique(val_labels)}\n")

# ========== Test Dataset ==========
test_features = df_test.drop(columns=['labels']).values.astype(np.float32)
test_labels = df_test['labels'].values.astype(np.int32)
test_dataset = tf.data.Dataset.from_tensor_slices(
    (test_features, test_labels)
).batch(batch_size)

print("=== Final Verification ===")
print(f"Total training clients: {len(train_datasets)}")
print(f"Test samples: {len(test_labels)}")
print(f"Test features shape: {test_features.shape}")

In [ ]:

# ## Cell5: Centralized Training (Label Skew Version)

import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

def centralized_training(seed=42):
    tf.keras.utils.set_random_seed(seed)

    # 1. Combine all client partitions (NEW FOR LABEL SKEW)
    full_train = pd.concat(data_partitions).sample(frac=1, random_state=seed)
    full_train_features = full_train.drop('labels', axis=1).values.astype(np.float32)
    full_train_labels = full_train['labels'].values.astype(np.int32)

    # 2. Create model (same architecture as FL)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(40,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')
    ])

    # 3. Training with metrics tracking
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

    history = model.fit(
        full_train_features, full_train_labels,
        epochs=30,
        batch_size=32,
        validation_data=(test_features, test_labels),  # Use your existing test data
        verbose=0
    )

    # 4. Final evaluation
    y_pred = np.argmax(model.predict(test_features), axis=1)

    return {
        'seed': seed,
        'train_loss': history.history['loss'],
        'val_loss': history.history['val_loss'],
        'test_accuracy': history.history['val_accuracy'][-1],
        'test_precision': precision_score(y_test, y_pred, average='macro', zero_division=0),
        'test_recall': recall_score(y_test, y_pred, average='macro', zero_division=0),
        'test_f1': f1_score(y_test, y_pred, average='macro', zero_division=0)
    }

# Run with multiple seeds
results_cl = [centralized_training(seed=s) for s in [42, 123, 456]]

# Generate report
report_cl = pd.DataFrame(results_cl)
print("\nCentralized Training Results (Label Skew Scenario):")
display(report_cl[['seed', 'test_accuracy', 'test_precision', 'test_recall', 'test_f1']])

In [ ]:
# ## Cell6: Federated Learning with Statistical Significance

import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import collections
from scipy import stats

# ======================
# 1. Data Preprocessing
# ======================

#Preprocessing function to handle batching with padding
def preprocess(dataset):
    def batch_format_fn(features, labels):
        return collections.OrderedDict(
            x=tf.reshape(features, [-1, 40]),  # Flatten features to match input shape
            y=tf.reshape(labels, [-1])  # Reshape labels to match expected shape
        )

    # Pad and batch datasets
    padded_shapes = ([None, 40], [None])  # Correct padded shapes for features and labels
    return dataset.padded_batch(32, padded_shapes=padded_shapes).map(batch_format_fn).prefetch(tf.data.experimental.AUTOTUNE)


# ======================
# 2. Core FL Functions
# ======================

def create_keras_model():
    return tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(40,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')
    ])

def run_fl_trial(seed=42):
    """Run complete FL pipeline with specified seed"""
    # Set all seeds
    tf.keras.utils.set_random_seed(seed)
    np.random.seed(seed)

    # Rebuild fresh training process
    def model_fn():
        return tff.learning.models.from_keras_model(
            create_keras_model(),
            input_spec=federated_train_data[0].element_spec,
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
        )

    training_process = tff.learning.algorithms.build_weighted_fed_avg(
        model_fn,
        client_optimizer_fn=tff.learning.optimizers.build_adam(0.001),
        server_optimizer_fn=tff.learning.optimizers.build_adam(0.01)
    )

    # Training loop
    state = training_process.initialize()
    for _ in range(30):
        state = training_process.next(state, federated_train_data).state

    # Evaluation
    eval_model = create_keras_model()
    eval_model.set_weights(list(training_process.get_model_weights(state).trainable))
    y_pred = np.argmax(eval_model.predict(test_features), axis=1)

    return {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='macro', zero_division=0),
        'recall': recall_score(y_test, y_pred, average='macro', zero_division=0),
        'f1': f1_score(y_test, y_pred, average='macro', zero_division=0)
    }

# ======================
# 3. Execution & Analysis
# ======================

# First create federated data (once)
NUM_CLIENTS = 10
federated_train_data = make_federated_data(train_datasets, list(range(NUM_CLIENTS)))

# Run multiple FL trials
fl_seeds = [42, 123, 456]
fl_results = [run_fl_trial(seed=s) for s in fl_seeds]
